# Importación de librerías

In [282]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torch.nn.functional as F
import torch.nn as nn

# Carga de datos

In [283]:
irisDataset = pd.read_csv("iris.data", names=["sepallength","sepalwidth","petallength","petalwidth","class"])

# Clase para escalar los datos (StandardScaler)

In [284]:
class StandardScaler:

    def __init__(self, mean=None, std=None, epsilon=1e-7):
        """Standard Scaler.
        The class can be used to normalize PyTorch Tensors using native functions. The module does not expect the
        tensors to be of any specific shape; as long as the features are the last dimension in the tensor, the module
        will work fine.
        :param mean: The mean of the features. The property will be set after a call to fit.
        :param std: The standard deviation of the features. The property will be set after a call to fit.
        :param epsilon: Used to avoid a Division-By-Zero exception.
        """
        self.mean = mean
        self.std = std
        self.epsilon = epsilon

    def fit(self, values):
        dims = list(range(values.dim() - 1))
        self.mean = torch.mean(values, dim=dims)
        self.std = torch.std(values, dim=dims)

    def transform(self, values):
        return (values - self.mean) / (self.std + self.epsilon)

    def fit_transform(self, values):
        self.fit(values)
        return self.transform(values)

    def __repr__(self):
        return f"mean: {self.mean}, std:{self.std}, epsilon:{self.epsilon}"

# Datasets y Dataloaders

In [285]:
class IrisDataset(Dataset):
  def __init__(self, src_file, root_dir, transform=None):
    irisDataset = pd.read_csv(src_file, names=["sepallength","sepalwidth","petallength","petalwidth","class"])

    X = irisDataset[irisDataset.columns.intersection(["sepallength","sepalwidth","petallength","petalwidth"])]
    Y = irisDataset[irisDataset.columns.intersection(["class"])]

    nomeClases = Y["class"].unique()
    conversion = {v: k for k, v in dict(enumerate(nomeClases)).items()}
    YConversion = pd.DataFrame()

    for nome in nomeClases:
      YConversion[nome] = (Y["class"]==nome).apply(lambda x : 1.0 if x else 0.0)
      
    y_tensor = torch.as_tensor(YConversion.to_numpy()).type(torch.float32)

    df_dict = dict.fromkeys(X.columns, '')
    X.rename(columns = df_dict)
    s1=X.iloc[:,0:4].values
    x_tensor = torch.tensor(s1)

    scaler = StandardScaler()
    
    XScalada = scaler.fit_transform(x_tensor).type(torch.float32)
    
    self.data = torch.cat((XScalada,y_tensor),1)
    self.root_dir = root_dir
    self.transform = transform
  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    preds = self.data[idx, 0:4]
    spcs = self.data[idx, 4:]
    sample = (preds, spcs)
    
    if self.transform:
      sample = self.transform(sample)
    return sample

# Prueba de carga de datos

In [286]:
dataset = IrisDataset("iris.data",".")
display(dataset[0]) 

(tensor([-0.8977,  1.0286, -1.3368, -1.3086]), tensor([1., 0., 0.]))

# División en train y test

In [287]:
lonxitudeDataset = len(dataset)

tamTrain =int(lonxitudeDataset*0.8)
tamVal = int(lonxitudeDataset*0.2)

print(f"Tam dataset: {lonxitudeDataset} train: {tamTrain} tamVal: {tamVal}")

train_set, val_set = random_split(dataset,[tamTrain,tamVal])

train_ldr = torch.utils.data.DataLoader(train_set, batch_size=2,
    shuffle=True, drop_last=False)

validation_loader =torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False, num_workers=2)

Tam dataset: 150 train: 120 tamVal: 30


# Creación del modelo

In [288]:
class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 50)
        self.layer3 = nn.Linear(in_features=50, out_features=3)
        
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

# Instanciación del modelo 

In [289]:
model     = Model(4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.CrossEntropyLoss()
display(model)

Model(
  (layer1): Linear(in_features=4, out_features=50, bias=True)
  (layer2): Linear(in_features=50, out_features=50, bias=True)
  (layer3): Linear(in_features=50, out_features=3, bias=True)
)

In [290]:
entradaProba,dest = next(iter(train_ldr))

print("Entrada:")
display(entradaProba)

print("Desexada:")
display(dest)

saida = model(entradaProba) # esta é a proba de verdade

print("Saída:")
display(saida)

loss_fn(saida, dest)

Entrada:


tensor([[-1.2600,  0.7980, -1.2234, -1.3086],
        [-0.0523,  2.1818, -1.4501, -1.3086]])

Desexada:


tensor([[1., 0., 0.],
        [1., 0., 0.]])

Saída:


tensor([[0.3903, 0.2545, 0.3552],
        [0.4083, 0.2767, 0.3151]], grad_fn=<SoftmaxBackward0>)

tensor(1.0343, grad_fn=<DivBackward1>)

# Función entrenamiento

In [291]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    
    # usamos enumerate para saber en que batch imos
    for i, data in enumerate(train_ldr):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()
        
        # Gather data and report
        running_loss += loss.item()
        
        if i % 10 == 9:
            last_loss = running_loss / 10 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.
            
    return last_loss

In [292]:
EPOCHS = 100
loss_list     = torch.zeros((EPOCHS,))
accuracy_list = torch.zeros((EPOCHS,))

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))

    # Poñemos o modelo en modo entrenamento
    model.train(True)
    avg_loss = train_one_epoch(epoch, None)
    loss_list[epoch] = avg_loss
    
    # Non se precisan os gradientes para o test
    model.train(False)

    running_vloss = 0.0
    for i, vdata in enumerate(validation_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)

        correct = (torch.argmax(voutputs, dim=0) == vlabels).type(torch.FloatTensor)
        accuracy_list[epoch] += correct.sum()
        running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {} {}/{}\n'.format(avg_loss, avg_vloss,accuracy_list[epoch],int(lonxitudeDataset*0.2)))
    

EPOCH 1:
  batch 10 loss: 1.059256410598755
  batch 20 loss: 1.0138394832611084
  batch 30 loss: 0.9566249966621398
  batch 40 loss: 0.9900769889354706
  batch 50 loss: 0.95292147397995
  batch 60 loss: 0.9077862739562989
LOSS train 0.9077862739562989 valid 0.9264757037162781 27.0/30

EPOCH 2:
  batch 10 loss: 0.8426261305809021
  batch 20 loss: 0.9216407835483551
  batch 30 loss: 0.8267547905445098
  batch 40 loss: 0.7912863552570343
  batch 50 loss: 0.73655064702034
  batch 60 loss: 0.7452371716499329
LOSS train 0.7452371716499329 valid 0.8365085124969482 29.0/30

EPOCH 3:
  batch 10 loss: 0.7483673453330993
  batch 20 loss: 0.7532182693481445
  batch 30 loss: 0.7983941018581391
  batch 40 loss: 0.7671001315116882
  batch 50 loss: 0.7079979538917541
  batch 60 loss: 0.693513149023056
LOSS train 0.693513149023056 valid 0.7696833610534668 23.0/30

EPOCH 4:
  batch 10 loss: 0.6920497179031372
  batch 20 loss: 0.784494012594223
  batch 30 loss: 0.7066557586193085
  batch 40 loss: 0.70421

In [293]:
torch.save(model.state_dict(), "pesos.pt")